<a href="https://colab.research.google.com/github/luciedum-dot/Intro-to-data-analysis-with-Python---Final-Project/blob/Branche-lucie/Evolutions_of_Criminality_France_2016_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

#Data Cleaning notebook
This notebook will be used for the first part of our project, which is data cleaning


In [ ]:
!pip install pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
!git clone https://github.com/luciedum-dot/Intro-to-data-analysis-with-Python---Final-Project.git



# **Part 1: Has criminality been on a rise in the past years?**

##1.1 Read and pivot Excel File - mec

In [ ]:
#Read Excel File - total accused people and display the sheets' names

path = "Intro-to-data-analysis-with-Python---Final-Project/Data/mec.xlsx"

xls = pd.ExcelFile(path)
xls.sheet_names


In [ ]:
#Show the Excel File on the sheet sex, in order to look at the "total" line for every year and see the trend

#Read Excel on the sheet 3.sexe, skipping the first 2 rows and setting multiple headers (years and categories)
df = pd.read_excel(path, sheet_name="3.sexe", skiprows=2, header=[0, 1])

#Remove the 1st empty column of the sheet
df = df.iloc[:,1:]

#Pivot the year header
pivoted_df = (
    df
    .set_index("Sexe")
    .stack(level=1, future_stack=True)          # move second header to rows & future_stack=True just allow us to avoid a warning
    .reset_index()           # make it a column
    .rename(columns={"level_1": "Année"})
  )

#Cast the type of Sexe column from tupple to string
pivoted_df["Sexe"] = pivoted_df["Sexe"].str[0]

pivoted_df.head(30)

##Display the df-mec just for Sex=Total

In [ ]:
#Visualize just the total column
#An other way to do it, but less reliable :df_total = pivoted_df.loc[18:26] #df_total.head(10)
df_total = pivoted_df[pivoted_df["Sexe"] == "Total"]
display(df_total)


##Plotting absolute evolution of homicides

In [ ]:
#Plot the absolute evolution of homicides to vizualize the trend
plt.plot(df_total['Année'], df_total['Homicides (y compris coups et blessures volontaires suivis de mort)'], '-oc')
plt.title("Evolution of the number of homicides between 2016-2024")
plt.xlabel("Year")
plt.ylabel("Number of homicides")
plt.show()

##Read Population Excel

In [ ]:
#Analyze the evolution of criminality with taking into account population growth:
path = "Intro-to-data-analysis-with-Python---Final-Project/Data/pop.xlsx"

xls_pop = pd.ExcelFile(path)
xls_pop.sheet_names



In [ ]:
#Show the Excel File on the sheet France - Figure 2, in order to look at the "Ensemble" column for every year between 2016 to 2024
df_pop = pd.read_excel(path, sheet_name="France - Figure 2", skiprows=3)
df_pop = df_pop[0:-4] #Remove the last 4 rows which contained strings
df_pop.head(10)


In [ ]:
df_pop_années = df_pop.loc[1:9]
display(df_pop_années)

##Plotting population evolution

In [ ]:
#Plot the evolution of the pop to vizualize the trend
plt.plot(df_pop_années['Année'], df_pop_années['Ensemble'], '-oc')
plt.title("Evolution of the population between 2016-2024")
plt.xlabel("Year")
plt.ylabel("Population (in thousands)")
plt.show()

##Add total pop on the same df as mec and plot their parallel evolution

In [ ]:
#Gather data on homicides and population on the same table to then plot their evolution on the same graph
df_total_hom = df_total[['Année', 'Homicides (y compris coups et blessures volontaires suivis de mort)']]
df_pop_années_ensemble = df_pop_années[['Année', 'Ensemble']]
df_merged = pd.merge(df_total_hom, df_pop_années_ensemble, on="Année")
df_merged_renamed = df_merged.rename(columns={"Ensemble": "Population"})
df_merged_renamed.head(10)

In [ ]:
#Plot the evolution of population and homicides on the same graph
fig, ax1 = plt.subplots()

# Population (left axis)
ax1.plot(
    df_merged_renamed['Année'],
    df_merged_renamed['Population'],
    marker='o',
    linestyle='-',
    color='blue',
    label='Population'
)
ax1.set_xlabel("Year")
ax1.set_ylabel("Population (in thousands)", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Homicides (right axis)
ax2 = ax1.twinx()
ax2.plot(
    df_merged_renamed['Année'],
    df_merged_renamed['Homicides (y compris coups et blessures volontaires suivis de mort)'],
    marker='s',
    linestyle='--',
    color='red',
    label='Number of homicides'
)
ax2.set_ylabel("Number of homicides", color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Title
plt.title("Evolution of population and homicides (2016–2024)")

# Grid
ax1.grid(True)

plt.show()

##1.2 Evolution offense's type


###1.2.1 Homicide

Compute the homicide rate

In [ ]:
#Calculate the homicide rate and plot it. The population number has been multiplied by 1000 as it was displayed in thousands people in the excelfile and the dataframes before
df_hom_rate = df_merged_renamed['Homicides (y compris coups et blessures volontaires suivis de mort)'] / (df_merged_renamed['Population']*1000) * 100_000
plt.plot(df_merged_renamed['Année'], df_hom_rate, marker='o')
plt.title("Homicide rate per 100,000 inhabitants (2016–2024)")
plt.xlabel("Year")
plt.ylabel("Homicide rate")
plt.grid(True)
plt.show()



###1.2.2 Sexual violence

Plotting absolute evolution of sexual violences and plot the sexual violences rate

In [ ]:
#Plot the absolute evolution of sexual violence to vizualize the trend
plt.plot(df_total['Année'], df_total['Violences sexuelles'], '-oc')
plt.title("Evolution of the number of sexual violences between 2016-2024")
plt.xlabel("Year")
plt.ylabel("Number of sexual violences")
plt.show()

In [ ]:
#Gather data on sexual violence and population on the same table to then plot their evolution on the same graph
df_total_sv = df_total[['Année', 'Violences sexuelles']]
df_merged_sv = pd.merge(df_total_sv, df_pop_années_ensemble, on="Année")
df_merged_sv_renamed = df_merged_sv.rename(columns={"Ensemble": "Population"})
df_merged_sv_renamed.head(10)

In [ ]:
#Calculate the sexual violence rate and plot it. The population number has been multiplied by 1000 as it was displayed in thousands people in the excelfile and the dataframes before
df_sv_rate = df_merged_sv_renamed['Violences sexuelles'] / (df_merged_sv_renamed['Population']*1000) * 100_000
plt.plot(df_merged_sv_renamed['Année'], df_sv_rate, marker='o')
plt.title("Sexual violence rate per 100,000 inhabitants (2016–2024)")
plt.xlabel("Year")
plt.ylabel("Sexual violence rate")
plt.grid(True)
plt.show()

###1.2.3 Physical violence

####Plotting the absolute evolution of physical domestic violence and its rate

In [ ]:
#Plot the absolute evolution of physical domestic violence to vizualize the trend
plt.plot(df_total['Année'], df_total['Violences physiques dans le cadre intra-familial'], '-oc')
plt.title("Evolution of the number of physical domestic violence between 2016-2024")
plt.xlabel("Year")
plt.ylabel("Number of physical domestic violences")
plt.show()

In [ ]:
#Gather data on domestic physical violence and population on the same table to then plot their evolution on the same graph
df_total_dpv = df_total[['Année', 'Violences physiques dans le cadre intra-familial']]
df_merged_dpv = pd.merge(df_total_dpv, df_pop_années_ensemble, on="Année")
df_merged_dpv_renamed = df_merged_dpv.rename(columns={"Ensemble": "Population"})
df_merged_dpv_renamed.head(10)

In [ ]:
#Calculate the domestic physical violence rate and plot it. The population number has been multiplied by 1000 as it was displayed in thousands people in the excelfile and the dataframes before
df_dpv_rate = df_merged_dpv_renamed['Violences physiques dans le cadre intra-familial'] / (df_merged_dpv_renamed['Population']*1000) * 100_000
plt.plot(df_merged_dpv_renamed['Année'], df_dpv_rate, marker='o')
plt.title("Domestic physical violence rate per 100,000 inhabitants (2016–2024)")
plt.xlabel("Year")
plt.ylabel("Domestic physical violence rate")
plt.grid(True)
plt.show()

####Plotting the absolute evolution of non-violent theft and its rate

In [ ]:
#Plot the absolute evolution of non-violent thefts to vizualize the trend
plt.plot(df_total['Année'], df_total['Vols sans violence contre des personnes'], '-oc')
plt.title("Evolution of the number of non-violent thefts between 2016-2024")
plt.xlabel("Year")
plt.ylabel("Number of non-violent theft")
plt.show()

In [ ]:
#Gather data on non-violent thefts and population on the same table to then plot their evolution on the same graph
df_total_nvt = df_total[['Année', 'Vols sans violence contre des personnes']]
df_merged_nvt = pd.merge(df_total_nvt, df_pop_années_ensemble, on="Année")
df_merged_nvt_renamed = df_merged_nvt.rename(columns={"Ensemble": "Population"})
df_merged_nvt_renamed.head(10)

In [ ]:
#Calculate the non-violent thefts rate and plot it. The population number has been multiplied by 1000 as it was displayed in thousands people in the excelfile and the dataframes before
df_nvt_rate = df_merged_nvt_renamed['Vols sans violence contre des personnes'] / (df_merged_nvt_renamed['Population']*1000) * 100_000
plt.plot(df_merged_nvt_renamed['Année'], df_nvt_rate, marker='o')
plt.title("Non-violent thefts rate per 100,000 inhabitants (2016–2024)")
plt.xlabel("Year")
plt.ylabel("Non-violent thefts rate")
plt.grid(True)
plt.show()

# **Part 2: What variables are most closely linked to committing a crime?**


## Sex

In [ ]:
#Reshaping the population data in long format (vs. wide format, where each category is in a separate column),
#so that each variable is in a single column, and there’s a column that indicates the category:
df_pop_années_long = df_pop_années.melt(
    id_vars='Année',
    value_vars=['Femmes', 'Hommes', 'Ensemble'],
    var_name='Sexe',
    value_name='Population'
)
df_pop_années_long.head()

In [ ]:
#Mapping the different sex so that the slight variations in the different df do not affect the implementation of the code

sex_mapping = {
    'Femmes': 'female',
    'Femme': 'female',
    'Hommes': 'male',
    'Homme': 'male',
    'Ensemble': 'total',
    'Total': 'total'
    }


In [ ]:
#Safety step: inspection code to check which values exist in the Sexe column in two different DataFrames (including NaN):

print(df_pop_années_long['Sexe'].value_counts(dropna=False))
print(pivoted_df['Sexe'].value_counts(dropna=False))

In [ ]:
#Copy created to not have to go on the top of the page to run the pivoted_df cell
pivoted_df_copy = pivoted_df.copy()


In [ ]:
#Standardization of the values in the Sexe column using the precedently created mapping dictionary:

df_pop_années_long['Sexe'] = df_pop_années_long['Sexe'].map(sex_mapping)

pivoted_df_copy['Sexe'] = pivoted_df_copy['Sexe'].map(sex_mapping)

### Sexual violences rate per 100k by sex

In [ ]:
#Computation of the totals (for the whole period 2016-2024) by sex (Sexe) from the two different DataFrames, on the variable violences sexuelles and on the variable population

sv_totals = (
    pivoted_df_copy
    .groupby('Sexe')['Violences sexuelles']
    .sum()
    .reset_index()
)

pop_totals = (
    df_pop_années_long
    .groupby('Sexe')['Population']
    .sum()
    .reset_index()
)

In [ ]:
#Compute the rate by sex for 100 000 individuals:

df_sv_rates_by_sex = pd.merge(sv_totals, pop_totals, on='Sexe')

df_sv_rates_by_sex['Rate per 100k'] = (
    df_sv_rates_by_sex['Violences sexuelles'] / (df_sv_rates_by_sex['Population']*1000) * 100_000
)
df_sv_rates_by_sex_renamed = df_sv_rates_by_sex.rename(columns={
    "Violences sexuelles": "Sexual violences",
    "Sexe": "Sex"
    })
df_sv_rates_by_sex_renamed

In [ ]:
#Plot barplots for different crime on the sex variable to see the criminality rate by sex,
# and then analyze for exemple which crimes women are the most likely to commit

plt.figure(figsize=(7,5))
plt.bar(df_sv_rates_by_sex_renamed["Sex"], df_sv_rates_by_sex_renamed["Rate per 100k"], color='skyblue', label='Sexual violences')
plt.title("Average sexual violence rate by sex (2016–2024)")
plt.xlabel("Sex")
plt.ylabel("Average rate for 2016-2024")
plt.legend()
plt.show()





###Homicides rate by sex

In [ ]:
#Computation of the totals by sex (Sexe) from the two different DataFrames, on the variable Homicides and on the variable population

hom_totals = (
    pivoted_df_copy
    .groupby('Sexe')['Homicides (y compris coups et blessures volontaires suivis de mort)']
    .sum()
    .reset_index()
)

pop_totals = (
    df_pop_années_long
    .groupby('Sexe')['Population']
    .sum()
    .reset_index()
)

In [ ]:
#Compute the rate by 100 000 individuals:

df_hom_rates_by_sex = pd.merge(hom_totals, pop_totals, on='Sexe')

df_hom_rates_by_sex['Rate per 100k'] = (
    df_hom_rates_by_sex['Homicides (y compris coups et blessures volontaires suivis de mort)'] / (df_hom_rates_by_sex['Population']*1000) * 100_000
)
df_hom_rates_by_sex_renamed = df_hom_rates_by_sex.rename(columns={
    "Homicides (y compris coups et blessures volontaires suivis de mort)": "Homicides",
    "Sexe": "Sex"
    })
df_hom_rates_by_sex_renamed

In [ ]:
#Plot barplots for different crime on the sex variable to see the criminality rate by sex,
# and then analyze for exemple which crimes women are the most likely to commit

plt.figure(figsize=(7,5))
plt.bar(df_hom_rates_by_sex_renamed["Sex"], df_hom_rates_by_sex_renamed["Rate per 100k"], color='skyblue', label='Homicides')
plt.title("Average homicides rate by sex (2016–2024)")
plt.xlabel("Sex")
plt.ylabel("Average rate for 2016-2024")
plt.legend()
plt.show()

### Non-violent theft rate by sex

In [ ]:

#Computation of the totals (for the whole period 2016-2024) by sex (Sexe) from the two different DataFrames, on the Vols sans violence contre des personnes and on the variable population

sv_totals = (
    pivoted_df_copy
    .groupby('Sexe')['Vols sans violence contre des personnes']
    .sum()
    .reset_index()
)

pop_totals = (
    df_pop_années_long
    .groupby('Sexe')['Population']
    .sum()
    .reset_index()
)


## Nationality

In [ ]:
#Show the Excel File on the sheet nationality, in order to look at the "total" line for every year and see the trend

#Read Excel on the sheet 2.nationalité, skipping the first 2 rows and setting multiple headers (years and categories)
df_nationality = pd.read_excel(path, sheet_name="2.nationalité", skiprows=2, header=[0, 1])

#Remove the 1st empty column of the sheet
df_nationality = df_nationality.iloc[:,1:]

#Pivot the year header
pivoted_nationality_df = (
    df_nationality
    .set_index("Nationalité")
    .stack(level=1, future_stack=True)          # move second header to rows & future_stack=True just allow us to avoid a warning
    .reset_index()           # make it a column
    .rename(columns={"level_1": "Année"})
  )

#Cast the type of Nationalité column from tupple to string
pivoted_nationality_df["Nationalité"] = pivoted_nationality_df["Nationalité"].str[0]

pivoted_nationality_df.head(15)

In [ ]:
#Analyze the evolution of criminality with taking into account change in population. A new excel has to be downloaded, showing the structure of population by nationality:
path = "Intro-to-data-analysis-with-Python---Final-Project/Data/pop_e.xlsx"

xls_pop_e = pd.ExcelFile(path)
xls_pop_e.sheet_names

In [ ]:
#Show the Excel File on the sheet Figure 5 – Population étrangère et immigrée en France, in order to look at the "Etrangers, effectif (en milliers)" column for every year between 2016 to 2024
df_pop_e = pd.read_excel(path, sheet_name="Figure 5", skiprows=2, header=[0, 1])
df_pop_e = df_pop_e[0:-5] #Remove the last 5 rows which contained strings
df_pop_e.head(10)

In [ ]:
df_pop_e_années = df_pop_e.loc[0:8]
display(df_pop_e_années)